In [ ]:
from chat_with_nerf.chat.agent import Agent 
import os
import json
import numpy as np
from chat_with_nerf.chat.session import Session
import time
import open3d as o3d
from pathlib import Path
from chat_with_nerf.chat.system_prompt import EDITED_SYSTEM_PROMPT, NO_VISUAL_FEEDBACK_SYSTEM_PROMPT
from chat_with_nerf.settings import Settings
from joblib import Parallel, delayed
from evaluation_vis_util import draw_plotly, create_bbox
from utils import is_label_unique, convert_origin_bbox, get_transformation_matrix, construct_bbox_corners, get_box3d_min_max, box3d_iou

In [ ]:
root_directory = ''  # Assuming current directory, adjust path if needed

In [ ]:
def get_val_set():
    json_dict = {}
    # List of all subfolders and their files
    subfolders_files = [(dp, filenames) for dp, _, filenames in os.walk(root_directory)]

    # Dictionary comprehension to pick only the first JSON from each subfolder
    json_dict = {os.path.basename(dp): os.path.join(dp, filenames[0]) for dp, filenames in subfolders_files if any(fn.endswith('.json') for fn in filenames)}

    return json_dict

# os.environ['API_URL'] = ""
# os.environ['OPENAI_API_KEY'] = """
os.environ['API_URL'] = ""
os.environ['OPENAI_API_KEY'] = ""
json_dict = get_val_set()

with open(scene_path, 'r') as file:
    data = json.load(file)
furnitures = data['objects']
alignment_matrix = Path("/workspace/chat-with-nerf-eval/data/scannet/scans")
specific_file_path = alignment_matrix / scene_name / (scene_name + ".txt")
axisAlignment_matrix = get_transformation_matrix(specific_file_path)

In [ ]:
acc_25 = 0
acc_50 = 0
acc_25_unique = 0
acc_50_unique = 0
acc_25_multiple = 0
acc_50_multiple = 0
list_iou = []
total_object = 0
session_id_list = []

In [ ]:
agent = Agent(scene_name=scene_name)
agent.API_URL = os.environ['API_URL']
agent.OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [ ]:
def process_description(scene_name, description, corners_original, is_unique, object_id, query_rank_id, center_original, extents_original):  
    MAX_RETRIES = 3
    for retry in range(MAX_RETRIES):
        try:
            new_session = Session.create_for_scene(scene_name)  
            new_session.session_id = f"{scene_name}-{object_id}-{query_rank_id}"  
            new_session.working_scene_name = scene_name
            new_session.grounding_query = description
            new_session.ground_truth = [center_original, extents_original]
            print(description)  
            generator = agent.act(  
                NO_VISUAL_FEEDBACK_SYSTEM_PROMPT,  
                description,  
                0.9,
                1,  
                scene_name,  
                new_session  
            )  
            for chat_history_for_display, chat_counter, get_status_code, session, grounding_result_mesh_path in generator:  
                pass  
            prediction = session.candidate[str(session.chosen_candidate_id)]  
            center = prediction['centroid']  
            extent = prediction['extent']  
            prediction = construct_bbox_corners(center, extent)  
            iou3d = box3d_iou(np.array(corners_original), prediction)  
            return iou3d, session.session_id, session, is_unique
        except Exception as exp:
            if retry < MAX_RETRIES - 1:  # If it's not the last retry
                print(f"Attempt {retry + 1} failed. Retrying...")
                continue
            else:  # On the last retry, print the exception and return a message
                print(exp)
                # return f"Failed after {MAX_RETRIES} attempts.", None, None, None

In [ ]:
preprocessed_data = []
for furniture in furnitures:
    bbox = furniture['bbox']
    center_original, extents_original = bbox[:3], bbox[3:6]
    corners_original = construct_bbox_corners(center_original, extents_original)
    label = furniture['label']
    is_unique = is_label_unique(furnitures, label)
    print(str(is_unique))
    
    for idx, description in enumerate(furniture['description']):
        preprocessed_data.append((scene_name, description, corners_original, is_unique, label, idx, center_original, extents_original))

In [ ]:
results = Parallel(n_jobs=15, backend='threading')(delayed(process_description)(*data) for data in preprocessed_data)

In [ ]:
for iou3d, session_id, session, is_unique in results:  
    if iou3d:
        if iou3d > 0.25:  
            acc_25 += 1
            if is_unique:  
                acc_25_unique += 1  
            else:
                acc_25_multiple += 1  
        if iou3d > 0.5:  
            acc_50 += 1  
            if is_unique:  
                acc_50_unique += 1  
            else:  
                acc_50_multiple += 1  
        list_iou.append(iou3d)
        session_id_list.append(session_id)  
        session.save()


In [ ]:
print("acc_25 =", acc_25)
print("acc_50 =", acc_50)
print("acc_25_unique =", acc_25_unique)
print("acc_50_unique =", acc_50_unique)
print("acc_25_multiple =", acc_25_multiple)
print("acc_50_multiple =", acc_50_multiple)
print("list_iou =", list_iou)
print("total_object =", total_object)
print("session_id_list =", session_id_list)